In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(832366, 109)



# Gaze and Intent

Is there a correlation between gaze and intent, i.e., if a learner sees bleeding, do they treat it?

In [4]:

mask_series = (frvrs_logs_df.action_type == 'PLAYER_GAZE')
df = frvrs_logs_df[mask_series]
df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T

,830652,837912,830118,837828
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE
elapsed_time,237426,98137,146464,86976
event_time,2023-09-11 14:21:09,2023-09-11 13:46:43,2023-09-11 14:19:38,2023-09-11 13:46:32
session_uuid,a3d6d913-7755-4e8d-a174-d5e491c4eac7,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,a3d6d913-7755-4e8d-a174-d5e491c4eac7,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
file_name,v.1.3/a3d6d913-7755-4e8d-a174-d5e491c4eac7.csv,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,v.1.3/a3d6d913-7755-4e8d-a174-d5e491c4eac7.csv,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv
logger_version,1.3,1.3,1.3,1.3
time_group,1,1,1,1
player_gaze_location,"(2.9, 0.0, 2.5)","(-0.5, 0.0, 10.0)","(2.9, 0.0, 2.5)","(2.0, 0.0, 7.0)"
player_gaze_patient_id,Tutorial Military Marine Root,Mike_13 Root,Tutorial Military Marine Root,Bob_1 Root
player_gaze_distance_to_patient,1.490099,1.016645,2.298676,1.228294


In [5]:

# Iterate over the patient ID and get all the bleeding
if nu.pickle_exists('bleeding_treated_df'):
    bleeding_treated_df = nu.load_object('bleeding_treated_df')
else:
    groupby_columns = ['session_uuid', 'time_group', 'patient_id']
    gb = frvrs_logs_df.sort_values(['elapsed_time']).groupby(groupby_columns)
    rows_list = []
    for (session_uuid, time_group, patient_id), df1 in gb:
        
        # Add the logger version and the groupby columns to the row dictionary
        row_dict = {}
        logger_version = df1.logger_version.unique().item()
        row_dict['logger_version'] = logger_version
        for cn in groupby_columns: row_dict[cn] = eval(cn)
        
        # Did the trainee gaze at this patient at least once?
        mask_series = (df1.action_type == 'PLAYER_GAZE')
        row_dict['gazed_at_patient'] = bool(df1[mask_series].shape[0])
        
        # Is this patient bleeding?
        mask_series = df1.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
        row_dict['is_patient_bleeding'] = bool(df1[mask_series].shape[0])
        
        # Was the bleeding treated?
        mask_series &= (df1.injury_treated_injury_treated == True)
        row_dict['bleeding_treated'] = bool(df1[mask_series].shape[0])
        
        # Add the row dictionary to the list
        rows_list.append(row_dict)
    
    # Create a data frame from the list of row dictionaries
    bleeding_treated_df = DataFrame(rows_list)
    nu.store_objects(bleeding_treated_df=bleeding_treated_df)

In [6]:

# Display the minority combinations
groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
df = nu.get_minority_combinations(bleeding_treated_df, groupby_columns)
display(df.T)

,9757,6392,3565,2958
logger_version,1.3,1.3,1.3,1.0
session_uuid,fde2c201-36fc-4123-bf42-f1bff1b24969,a3d6d913-7755-4e8d-a174-d5e491c4eac7,5d916938-e0d3-47ee-8f55-12bf12283ee1,4727e60f-7e08-4595-b565-b6e80b7a8d87
time_group,1,1,2,1
patient_id,Military Mike Jungle Combat_1_5 Root,Tutorial Military Marine Root,Lily_4 Root,Mike_5 Root
gazed_at_patient,True,True,False,False
is_patient_bleeding,False,True,True,True
bleeding_treated,False,True,False,True


In [7]:

groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
srs = bleeding_treated_df.groupby(groupby_columns).size()

# Get the count of cases where all conditions are met
count_true_true_true = srs.get((True, True, True), 0)

# Get the count of cases where gazed_at_patient is True and is_patient_bleeding is True
count_true_true = srs.get((True, True, False), 0) + srs.get((True, True, True), 0)

# Calculate the percentage
percentage = (count_true_true_true / count_true_true) * 100 if count_true_true > 0 else 0
if percentage == 100:
    yesno = 'Yes'
    percentage = f'all {count_true_true_true}'
elif percentage < 100:
    yesno = 'Not quite'
    percentage = f'{percentage:.1f}%'
elif percentage == 0:
    yesno = 'No'
    percentage = 'none'

print(f'{yesno}, {percentage} of the trainees who gazed at a patient and discovered they were bleeding, treated the patient.')

Yes, all 20 of the trainees who gazed at a patient and discovered they were bleeding, treated the patient.


In [11]:

srs.to_frame().rename(columns={0: 'task_count'}).reset_index(drop=False)

,gazed_at_patient,is_patient_bleeding,bleeding_treated,task_count
0,False,False,False,7881
1,False,True,False,25
2,False,True,True,1932
3,True,False,False,16
4,True,True,True,20
